# Lane-Based Data: merge and unstack

This is a demonstration of using python

1. Load and clean a dataset
2. Create a 100 metre segmentation of the road network
3. Merge data against the segmentation
4. Un-stack the data into wide (lane-based) format

## Get our tools installed

For this process you will need `Python 3.9` or later.

We will be useing a custom python package called `segmenter`
The following two cells will check that we have everything installed and ready to go:

In [63]:
import sys

print(f"You are running python {sys.version}.")

# confirm that the current version of python is 3.9 or later or throw an exception
if not sys.version_info.major >= 3 and sys.version_info.minor >= 9:
    raise Exception(f"This notebook requires python 3.9 or later.")

# print(f"Your python executable is here: {sys.executable}")

You are running python 3.9.1 (default, Dec 11 2020, 09:29:25) [MSC v.1916 64 bit (AMD64)].


In [61]:
# check the version of python that the terminal is talking to: Should match the version number above
!Powershell Write-Host "your terminal is configured to call the following python version"
!Powershell python --version
# Uncomment these lines for more information
# !Powershell Write-Host "Your python installation is here"
# !Powershell (where.exe python).Split([Environment]::NewLine)[0]
# !Powershell Write-Host "And your packages are installed in these locations"
# !python -m site

your terminal is configured to call the following python version
Python 3.9.1


In [73]:
# install editable version
!pip install -e ../ --quiet

In [72]:
# force uninstall of editable install
import os
from distutils.sysconfig import get_python_lib
path = os.path.join(get_python_lib(), "segmenter.egg-link")
if os.path.exists(path):
    os.unlink(path)
else:
    print("No egg-link file found")

In [64]:
# If this cell runs and no errors appear below. we should be good to go.
# Normally takes about 10-20 seconds
# remove the --quiet flags if you are having trouble to see more information as pip tries to do its thing
!pip uninstall segmenter -y --quiet
!pip install https://github.com/thehappycheese/segmenter/archive/refs/tags/v0.1.2.zip --quiet

In [1]:
from segmenter import split_rows_by_category_to_max_segment_length
import pandas as pd
import numpy as np

In [16]:
df = pd.read_csv("./test_data/02.01 - Surface Detail.ALL.2021-07-06.12-51.343776376.csv")

C:\Users\e81005\Miniconda39\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
# select rows where POE_TEXT is blank and XSP is not blank
df = df[df["POE_TEXT"].isna() | df["XSP"].isna()]

In [18]:
# define a map for columns to rename
surf_cwy_map = {
    "ROAD_NO":            "road_no",
    "CWAY":               "cwy",
    "XSP":                "xsp",
    "SLK_FROM":           "slk_from",
    "SLK_TO":             "slk_to",
    "TRUE_FROM":          "true_from",
    "TRUE_TO":            "true_to",
    "SURF_WIDTH":         "surface_width",
    "SURF_YEAR":          "surface_year",
    "SURF_TYPE":          "surface_type_name",
    "SURF_AGG_SIZE":      "surface_aggregate_size",
    "SURF_ASPHALT_DEPTH": "surface_asphalt_depth",
}

# keep only the columns in the left of the map above
df = df[surf_cwy_map.keys()]

# rename remaining columns using the map above
df = df.rename(columns=surf_cwy_map)

# preview the dataframe by writing the variable name df as the last line in this cell
df

,road_no,cwy,xsp,slk_from,slk_to,true_from,true_to,surface_width,surface_year,surface_type_name,surface_aggregate_size,surface_asphalt_depth
0,H001,L,L1,0.00,0.04,0.00,0.04,3.9,2000.0,Asphalt Intersection Mix,NaN,40.0
1,H001,L,L2,0.00,0.04,0.00,0.04,3.3,2000.0,Asphalt Intersection Mix,NaN,40.0
2,H001,L,L1,0.04,0.06,0.04,0.06,5.3,2019.0,Asphalt Dense Graded,NaN,40.0
3,H001,L,L2,0.04,0.06,0.04,0.06,3.9,2000.0,Asphalt Dense Graded,NaN,40.0
4,H001,L,L3,0.04,0.06,0.04,0.06,3.3,2000.0,Asphalt Dense Graded,NaN,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...
137643,H924,S,L1,6.80,7.80,6.80,7.80,3.5,2004.0,Single Seal,14 mm,NaN
137644,H924,S,R1,6.80,7.80,6.80,7.80,3.5,1992.0,Single Seal,14 mm,NaN
137645,H924,S,R,6.80,7.80,6.80,7.80,1.0,1992.0,Single Seal,14 mm,NaN
137646,H926,S,NaN,0.00,0.42,0.00,0.42,NaN,NaN,NaN,NaN,NaN


In [19]:
# create a list of XSP values we want to keep
keep_xsp = [
    'L1', 'L2', 'L3', 'L4', 'L5', 'L6',
    'R1', 'R2', 'R3', 'R4', 'R5', 'R6'
]

# discard rows where xsp is not in keep_xsp
df = df[df["xsp"].isin(keep_xsp)]

# check the result
df["xsp"].value_counts().sort_index()

L1    32227
L2     7034
L3     1173
L4      281
L5       73
L6       25
R1    30121
R2     6043
R3      906
R4      241
R5       41
Name: xsp, dtype: int64

### Obtain a 100 metre Segmentation of the data in long format

- Segments are made at integer multiples of 100 metres
- Segments shorter than 20metres will be combined with adjacent segments
- Other columns are brought along keeping the observation with the longest overlap with the new 100 metre segment

The structure of the output produced by this next cell is suitable for the merge tool

In [20]:
# this process typically takes 2.5 minutes for the state road network
# and produces ~374708 rows
segmentation = split_rows_by_category_to_max_segment_length(
    df,
    measure_slk        = ("slk_from", "slk_to"),
    measure_true       = ("true_from","true_to"),
    categories         = ["road_no", "cwy", "xsp"],
    max_segment_length = 0.100,
    min_segment_length = 0.010,
)
segmentation

slk_from  slk_to  true_from  true_to  \
road_no cwy xsp segment_index                                         
H001    L   L1  0                  0.00    0.10       0.00     0.10   
                0                  0.10    0.20       0.10     0.20   
                0                  0.20    0.24       0.20     0.24   
                1                  0.51    0.60       0.51     0.60   
                1                  0.60    0.70       0.60     0.70   
...                                 ...     ...        ...      ...   
M405    S   L1  2671               0.00    0.10       0.00     0.10   
                2671               0.10    0.20       0.10     0.20   
                2671               0.20    0.22       0.20     0.22   
M406    S   L1  2672               0.00    0.10       0.00     0.10   
                2672               0.10    0.19       0.10     0.19   

                               surface_asphalt_depth  surface_year  \
road_no cwy xsp segment_index                                        
H001    L   L1  0                               40.0        2000.0   
                0                                NaN        2000.0   
                0                                NaN        2000.0   
                1                                NaN        2000.0   
                1                                NaN        2000.0   
...                                              ...           ...   
M405    S   L1  2671                             NaN        2013.0   
                2671                             NaN        2013.0   
                2671                             NaN        2013.0   
M406    S   L1  2672                            40.0        2014.0   
                2672                            40.0        2014.0   

                              surface_aggregate_size  \
road_no cwy xsp segment_index                          
H001    L   L1  0                                NaN   
                0                                NaN   
                0                                NaN   
                1                                NaN   
                1                                NaN   
...                                              ...   
M405    S   L1  2671                     14 and 7 mm   
                2671                     14 and 7 mm   
                2671                     14 and 7 mm   
M406    S   L1  2672                             NaN   
                2672                             NaN   

                                      surface_type_name  surface_width  
road_no cwy xsp segment_index                                           
H001    L   L1  0              Asphalt Intersection Mix            3.9  
                0                  Asphalt Dense Graded            3.0  
                0                  Asphalt Dense Graded            3.0  
                1                  Asphalt Dense Graded            3.0  
                1                  Asphalt Dense Graded            3.0  
...                                                 ...            ...  
M405    S   L1  2671                      Two Coat Seal            3.5  
                2671                      Two Coat Seal            3.5  
                2671                      Two Coat Seal            3.5  
M406    S   L1  2672           Asphalt Intersection Mix            8.0  
                2672           Asphalt Intersection Mix            8.0  

[374708 rows x 9 columns]

In [21]:
segmentation.to_csv(    "../delme/segmentation_state_network.csv")
segmentation.to_parquet("../delme/segmentation_state_network.parquet")

In [27]:
segmentation = pd.read_parquet("../delme/segmentation_state_network.parquet")

In [28]:
# here we take a fresh copy of our segmentation dataframe,
# this way we can edit and re-run this cell many times 
# without accedentally re-manipulating mangled data
seg = segmentation.copy()

# Convert linear measures to integer metres;
# otherwise rounding errors cause strange behaviour when we  call .unstack("xsp").
# Note: We ask pandas to use the type "u4", which is an unsigned 32 bit integer, (can hold positive values up to 2^32 or about 4 Billion)
# pandas will typically use "f8" by default, which is a 64 bit floating point number.
# floating point numbers cause problems with comparisons; For example, the computer considers 0.1 to be exactly equal to 0.1000000000000000055
seg["slk_from"]  = (seg["slk_from"]  * 1000).round().astype("u4")
seg["slk_to"]    = (seg["slk_to"]    * 1000).round().astype("u4")
seg["true_from"] = (seg["true_from"] * 1000).round().astype("u4")
seg["true_to"]   = (seg["true_to"]   * 1000).round().astype("u4")

# Remove all columns from the hierarchical row index created by the `split_rows_by_category_to_max_segment_length()` function
seg = seg.reset_index()

# Create a new column called "dirn" which will be the first ("zeroth") character of the xsp column (eg. "L1" -> "L")
seg["dirn"] = seg["xsp"].str[0]

# Create a new column called "lane" which will be the second ("oneth") character of the xsp column (eg. "L1" -> "1")
seg["lane"] = seg["xsp"].str[1]

# Optionally drop the xsp column since it is now redundant
seg = seg.drop(columns="xsp")

# Create a new hierarchical row index for the segmentation
# For "unstacking" to be possible the row index must be unique for each row when "lane" is removed from the index
# "carriageway", and "true_to" are optional, but at least ["road_no", "dirn", "true_from", "slk_from", "slk_to", "lane"] are required
seg = seg.set_index(["road_no", "cwy", "dirn", "true_from", "true_to", "slk_from", "slk_to", "lane"])

# preview seg
# note the junk columns:
# - "segment_index" and "original_index" are leftovers from the `split_rows_by_category_to_max_segment_length()` function. They are left in as they are useful for other purposes.
# - All the other columns from the original surface details dataframe are still here (eg surface aggregate size, surface_year etc).
#   We are not really interested in them, lets just ignore them for now.
seg

segment_index  \
road_no cwy dirn true_from true_to slk_from slk_to lane                  
H001    L   L    0         100     0        100    1                 0   
                 100       200     100      200    1                 0   
                 200       240     200      240    1                 0   
                 510       600     510      600    1                 1   
                 600       700     600      700    1                 1   
...                                                                ...   
M405    S   L    0         100     0        100    1              2671   
                 100       200     100      200    1              2671   
                 200       220     200      220    1              2671   
M406    S   L    0         100     0        100    1              2672   
                 100       190     100      190    1              2672   

                                                         surface_asphalt_depth  \
road_no cwy dirn true_from true_to slk_from slk_to lane                          
H001    L   L    0         100     0        100    1                      40.0   
                 100       200     100      200    1                       NaN   
                 200       240     200      240    1                       NaN   
                 510       600     510      600    1                       NaN   
                 600       700     600      700    1                       NaN   
...                                                                        ...   
M405    S   L    0         100     0        100    1                       NaN   
                 100       200     100      200    1                       NaN   
                 200       220     200      220    1                       NaN   
M406    S   L    0         100     0        100    1                      40.0   
                 100       190     100      190    1                      40.0   

                                                         surface_year  \
road_no cwy dirn true_from true_to slk_from slk_to lane                 
H001    L   L    0         100     0        100    1           2000.0   
                 100       200     100      200    1           2000.0   
                 200       240     200      240    1           2000.0   
                 510       600     510      600    1           2000.0   
                 600       700     600      700    1           2000.0   
...                                                               ...   
M405    S   L    0         100     0        100    1           2013.0   
                 100       200     100      200    1           2013.0   
                 200       220     200      220    1           2013.0   
M406    S   L    0         100     0        100    1           2014.0   
                 100       190     100      190    1           2014.0   

                                                        surface_aggregate_size  \
road_no cwy dirn true_from true_to slk_from slk_to lane                          
H001    L   L    0         100     0        100    1                      None   
                 100       200     100      200    1                      None   
                 200       240     200      240    1                      None   
                 510       600     510      600    1                      None   
                 600       700     600      700    1                      None   
...                                                                        ...   
M405    S   L    0         100     0        100    1               14 and 7 mm   
                 100       200     100      200    1               14 and 7 mm   
                 200       220     200      220    1               14 and 7 mm   
M406    S   L    0         100     0        100    1                      None   
                 100       190     100      190    1                      None   

          

### Ready to Merge

At this point we are ready to merge additional datasets against our segmentation

> TODO:
> 
> Build a merge example.
>
> For now, imagine we added a columns such as `"IRI"` or `"Curv"` or `"Defl"` to the `seg` dataframe previewed above.<br>
> 
> For the following example we will just pretend we are interested in `"surface_width"` and `"surface_year"`

### Ready to Unstack

If we merged additional data columns in the previous step we are ready to unstack our data.
This is a trivial operation in excel with pivot tables, but since we are all set up here in python
lets go ahead with it here:

In [29]:
# select the columns we are "interested" in un-stacking:
seg = seg[["surface_width", "surface_year"]]

#preview seg
seg

surface_width  \
road_no cwy dirn true_from true_to slk_from slk_to lane                  
H001    L   L    0         100     0        100    1               3.9   
                 100       200     100      200    1               3.0   
                 200       240     200      240    1               3.0   
                 510       600     510      600    1               3.0   
                 600       700     600      700    1               3.0   
...                                                                ...   
M405    S   L    0         100     0        100    1               3.5   
                 100       200     100      200    1               3.5   
                 200       220     200      220    1               3.5   
M406    S   L    0         100     0        100    1               8.0   
                 100       190     100      190    1               8.0   

                                                         surface_year  
road_no cwy dirn true_from true_to slk_from slk_to lane                
H001    L   L    0         100     0        100    1           2000.0  
                 100       200     100      200    1           2000.0  
                 200       240     200      240    1           2000.0  
                 510       600     510      600    1           2000.0  
                 600       700     600      700    1           2000.0  
...                                                               ...  
M405    S   L    0         100     0        100    1           2013.0  
                 100       200     100      200    1           2013.0  
                 200       220     200      220    1           2013.0  
M406    S   L    0         100     0        100    1           2014.0  
                 100       190     100      190    1           2014.0  

[374708 rows x 2 columns]

In [30]:
# .unstack() by "lane"
seg = seg.unstack("lane")

# preview seg
seg

surface_width            \
lane                                                           1    2    3   
road_no cwy dirn true_from true_to slk_from slk_to                           
H001    L   L    0         100     0        100              3.9  3.3  NaN   
                 40        80      40       80               NaN  NaN  3.0   
                 60        80      60       80               NaN  NaN  NaN   
                 100       200     100      200              3.0  3.3  NaN   
                 200       240     200      240              3.0  3.3  NaN   
...                                                          ...  ...  ...   
M405    S   L    0         100     0        100              3.5  NaN  NaN   
                 100       200     100      200              3.5  NaN  NaN   
                 200       220     200      220              3.5  NaN  NaN   
M406    S   L    0         100     0        100              8.0  NaN  NaN   
                 100       190     100      190              8.0  NaN  NaN   

                                                                surface_year  \
lane                                                  4   5   6            1   
road_no cwy dirn true_from true_to slk_from slk_to                             
H001    L   L    0         100     0        100     NaN NaN NaN       2000.0   
                 40        80      40       80      NaN NaN NaN          NaN   
                 60        80      60       80      3.3 NaN NaN          NaN   
                 100       200     100      200     NaN NaN NaN       2000.0   
                 200       240     200      240     NaN NaN NaN       2000.0   
...                                                 ...  ..  ..          ...   
M405    S   L    0         100     0        100     NaN NaN NaN       2013.0   
                 100       200     100      200     NaN NaN NaN       2013.0   
                 200       220     200      220     NaN NaN NaN       2013.0   
M406    S   L    0         100     0        100     NaN NaN NaN       2014.0   
                 100       190     100      190     NaN NaN NaN       2014.0   

                                                                            \
lane                                                     2       3       4   
road_no cwy dirn true_from true_to slk_from slk_to                           
H001    L   L    0         100     0        100     2000.0     NaN     NaN   
                 40        80      40       80         NaN  2000.0     NaN   
                 60        80      60       80         NaN     NaN  2000.0   
                 100       200     100      200     2000.0     NaN     NaN   
                 200       240     200      240     2000.0     NaN     NaN   
...                                                    ...     ...     ...   
M405    S   L    0         100     0        100        NaN     NaN     NaN   
                 100       200     100      200        NaN     NaN     NaN   
                 200       220     200      220        NaN     NaN     NaN   
M406    S   L    0         100     0        100        NaN     NaN     NaN   
                 100       190     100      190        NaN     NaN     NaN   

                                                            
lane                                                 5   6  
road_no cwy dirn true_from true_to slk_from slk_to          
H001    L   L    0         100     0        100    NaN NaN  
                 40        80      40       80     NaN NaN  
                 60        80      60       80     NaN NaN  
                 100       200     100      200    NaN NaN  
                 200       240     200      240    NaN NaN  
...                                                 ..  ..  
M405    S   L    0         100     0        100    NaN NaN  
                 100       200     100      200    NaN NaN  
                 200       220     200      220    NaN NaN  
M406    S   

In [31]:
# do some final clean-up

# remove the hierarchical row index we used in the previous step
# (otherwise we cant sort by "road_number" since it is bound up in the index.)
seg = seg.reset_index()

# Sort things so that our data reads like we are driving down the road
seg = seg.sort_values(by=["road_no", "dirn", "true_from"])

# after the `.unstack()` we need to convert our measures back to kilometers
# this time we will ask for the type "f4", which is a 32 bit floating point number,
# capable of holding positive with about 5 decimal places of accuracy which is enough for any WA SLK
seg["slk_from"]  = seg["slk_from" ].astype("f4") / 1000.0
seg["slk_to"]    = seg["slk_to"   ].astype("f4") / 1000.0
seg["true_from"] = seg["true_from"].astype("f4") / 1000.0
seg["true_to"]   = seg["true_to"  ].astype("f4") / 1000.0

# preview our final output
# (NOTE: `NaN` cells will show up as blank in the output csv)
seg

road_no cwy dirn true_from true_to slk_from slk_to surface_width       \
lane                                                                  1    2   
0         H001   L    L      0.00    0.10     0.00   0.10           3.9  3.3   
1         H001   L    L      0.04    0.08     0.04   0.08           NaN  NaN   
2         H001   L    L      0.06    0.08     0.06   0.08           NaN  NaN   
3         H001   L    L      0.10    0.20     0.10   0.20           3.0  3.3   
4         H001   L    L      0.20    0.24     0.20   0.24           3.0  3.3   
...        ...  ..  ...       ...     ...      ...    ...           ...  ...   
354976    M405   S    L      0.00    0.10     0.00   0.10           3.5  NaN   
354977    M405   S    L      0.10    0.20     0.10   0.20           3.5  NaN   
354978    M405   S    L      0.20    0.22     0.20   0.22           3.5  NaN   
354979    M406   S    L      0.00    0.10     0.00   0.10           8.0  NaN   
354980    M406   S    L      0.10    0.19     0.10   0.19           8.0  NaN   

                         surface_year                                  
lane      3    4   5   6            1       2       3       4   5   6  
0       NaN  NaN NaN NaN       2000.0  2000.0     NaN     NaN NaN NaN  
1       3.0  NaN NaN NaN          NaN     NaN  2000.0     NaN NaN NaN  
2       NaN  3.3 NaN NaN          NaN     NaN     NaN  2000.0 NaN NaN  
3       NaN  NaN NaN NaN       2000.0  2000.0     NaN     NaN NaN NaN  
4       NaN  NaN NaN NaN       2000.0  2000.0     NaN     NaN NaN NaN  
...     ...  ...  ..  ..          ...     ...     ...     ...  ..  ..  
354976  NaN  NaN NaN NaN       2013.0     NaN     NaN     NaN NaN NaN  
354977  NaN  NaN NaN NaN       2013.0     NaN     NaN     NaN NaN NaN  
354978  NaN  NaN NaN NaN       2013.0     NaN     NaN     NaN NaN NaN  
354979  NaN  NaN NaN NaN       2014.0     NaN     NaN     NaN NaN NaN  
354980  NaN  NaN NaN NaN       2014.0     NaN     NaN     NaN NaN NaN  

[354981 rows x 19 columns]

In [33]:
# Finally, save the result to some file path of your choice:
seg.to_csv("../delme/RES.csv")